# Homework 4: Data-driven decison making

Read the case study \"*Select Collections, Inc.*\" included with this assignment. Your group will play a role similar to that of the summer intern, Marcos Kilduff. However, your assignment is not to predict *totalpay* for the observations in the test set. Instead, your task is to **decide** which of the accounts in the test set should be purchased and to **evaluate** the profits that your purchasing decisions could generate. Furthermore, assume that purchasing any invidiual account in the test set has a cost of **$800**.

## Grade Breakdown and Instructions

The grade breakdown for this assignment is as follows:
1. **Decision-making (60\%):** Your group will be evaluated based on how much profits your decisions generate compared to other groups in the class. So, the more profits you generate, the better. Your decisions must be uploaded in a file called "decisions_Group_no.csv" (name the file with your Group_no.). The first column must have the header "acctid" and contain the ID of all the accounts in the test set. The second column must have the header "decision" and contain the purchase decisions for all the accounts in the test set: use a 1 for accounts you decide to purchase and a 0 for accounts you decide not to purchase. For an example of how to output a file with these characteristics, see the code below. **IMPORTANT**: You will receive no credits if the file you upload does not follow these guidelines, so please be careful.
2. **Evaluation (40\%):** You must also report an estimate of how much profits do you expect to make with your decisions. You will be evaluated based on how well you estimate the profits you actually make, and as in the previous task, your grade will depend on the performance of other groups. Furthermore, overestimations of the actual profits will be penalized twice as much as underestimations. Your estimate must be uploaded as a non-formatted integer number in a file called "estimate_Group_no.txt" (name the file with your Group_no.).txt **IMPORTANT**: You will receive no credits if the file you upload does not follow these guidelines, so please be careful.

3. **This jupyter notebook** we will not look at your codings if your group performed well in the competition, if not, we shall look at your codings and see whetheter the logic behind is making sense so we can still give you partial marks. You can try to answer the questions in between.

* Zip the above 3 files and upload to Canvas, only one group member is required to upload.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [ ]:
# Load data
df_labeled = pd.read_csv("SC Training Data.csv")
df_unlabeled = pd.read_csv("SC Test Data.csv")

In [ ]:
df_labeled.head()

,acctid,state,zip,rollout,cobal,collscr,cs,accessscr,lnacscr,bureauscr,eaglemod,numcalls,numrpcs,totalpay
0,1,CA,93313,Wells,1786,4013181,4,0.1992,-1.61,133,67,104,3,360
1,2,CA,92154,Wells,2048,1015805,1,0.4376,-0.83,109,6,233,6,300
2,3,FL,33319,Wells,5170,4013057,4,0.8140,-0.21,103,90,28,1,10
3,4,WA,98115,Bank_Of_Am,4506,8012265,8,0.2978,-1.21,136,11,8,2,2550
4,5,MI,49615,Associates,2706,3013630,3,0.3722,-0.99,105,42,45,11,2000


Question 1: Before you begin, think about the business problem of the company (you can write your answer in the cell)

The business problem faced by Select Collections, Inc. is the need to optimize their collection processes for delinquent accounts. To achieve this, Select Collections needs to make data-driven decisions on which accounts to purchase and how much to pay for them. They want to build a predictive model that can forecast the amount of money they will collect from delinquent accounts. The ultimate goal is to improve the efficiency and effectiveness of the collection process, maximizing the total amount of payments received from the account holders. By accurately predicting the totalpay variable, Select Collections can make informed decisions regarding account acquisition and optimize their collection tactics and strategies.


## Step 1: Data Preparation

* Please conduct data preparation before fit the training data into models.
* Remember you have to do data preparation for both training and testing data

In [ ]:
# considering the zip and state share similar meaning in terms of the account holder living location, state is used, as zip is too specific.
#from the distribution graph/scatter plot of exhitbit 2, accessor, collscr, eaglemod shows no obvious pattern, thus will be dropped from the model
# as from data dictionary, collscr cant be interpreted, thus collscr and cs would be dropped.

df_labeled = df_labeled.drop(['accessscr', 'collscr', 'eaglemod', 'cs', 'lnacscr'], axis=1)
df_unlabeled = df_unlabeled.drop(['accessscr', 'collscr', 'eaglemod', 'cs', 'lnacscr'], axis=1)


In [ ]:
#check if there is missing value
missing_values_labeled = df_labeled.isnull().sum()
missing_values_unlabeled = df_unlabeled.isnull().sum()
print("labeled", missing_values_labeled)
print("unlabeled", missing_values_unlabeled)

labeled acctid       0
state        0
zip          0
rollout      0
cobal        0
bureauscr    0
numcalls     0
numrpcs      0
totalpay     0
dtype: int64
unlabeled acctid       0
state        0
zip          0
rollout      0
cobal        0
bureauscr    0
numcalls     0
numrpcs      0
dtype: int64


In [ ]:
#Handle categoricals
#labelled data
rollout_dummies = pd.get_dummies(df_labeled['rollout'], prefix='rollout')
state_dummies = pd.get_dummies(df_labeled['state'], prefix='rollout')
df_labeled = pd.concat([df_labeled, rollout_dummies, state_dummies], axis=1)
df_labeled.drop(['rollout', "state"], axis=1, inplace=True)

#unlabel data
rollout_dummies = pd.get_dummies(df_unlabeled['rollout'], prefix='rollout')
state_dummies = pd.get_dummies(df_unlabeled['state'], prefix='rollout')
df_unlabeled = pd.concat([df_unlabeled, rollout_dummies, state_dummies], axis=1)
df_unlabeled.drop(['rollout', "state"], axis=1, inplace=True)

In [ ]:
df_labeled

,acctid,zip,cobal,bureauscr,numcalls,numrpcs,totalpay,rollout_Associates,rollout_Bank_Of_Am,rollout_Chase,...,rollout_SD,rollout_TN,rollout_TX,rollout_UT,rollout_VA,rollout_VT,rollout_WA,rollout_WI,rollout_WV,rollout_WY
0,1,93313,1786,133,104,3,360,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,92154,2048,109,233,6,300,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,33319,5170,103,28,1,10,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,98115,4506,136,8,2,2550,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,5,49615,2706,105,45,11,2000,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3565,3566,44657,2301,110,41,4,1600,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3566,3567,90005,1763,125,6,2,50,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3567,3568,33810,4231,132,95,12,365,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3568,3569,20895,2123,121,48,3,728,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.preprocessing import MinMaxScaler
#Scaling Numerical Variables, consider the data is at different scales to prevent certain variables from dominating the distance calcuation
#eg: cobal

scaler = MinMaxScaler()
numerical_variables = df_labeled.columns.drop('totalpay')
df_labeled[numerical_variables] = scaler.fit_transform(df_labeled[numerical_variables])
df_labeled[numerical_variables] = scaler.fit_transform(df_labeled[numerical_variables])

numerical_variables_unlabeled = df_unlabeled.columns
df_unlabeled[numerical_variables] = scaler.fit_transform(df_unlabeled[numerical_variables])
df_unlabeled[numerical_variables] = scaler.fit_transform(df_unlabeled[numerical_variables])


In [ ]:
import pandas as pd
from scipy.stats import pearsonr

correlations = []
p_values = []
for feature in df_labeled.iloc[:, :-1]:
    correlation, p_value = pearsonr(df_labeled[feature], df_labeled['totalpay'])
    correlations.append(correlation)
    p_values.append(p_value)

# Create a DataFrame to store the correlation and p-values
feature_stats = pd.DataFrame({'Feature': df_labeled.columns[:-1], 'Correlation': correlations, 'P-value': p_values})

# Sort the DataFrame by absolute correlation in descending order
feature_stats = feature_stats.reindex(feature_stats['Correlation'].abs().sort_values(ascending=False).index)

# Set the display options to show all rows
pd.set_option('display.max_rows', 10)

# Print the correlation_stats DataFrame
print(feature_stats)

mean_value = feature_stats['Correlation'].mean()
mode_value = feature_stats['Correlation'].mode().values[0]
median_value = feature_stats['Correlation'].median()
sd_value = feature_stats['Correlation'].std()
min_value = feature_stats['Correlation'].min()
max_value = feature_stats['Correlation'].max()

print("Mean:", mean_value)
print("Mode:", mode_value)
print("Median:", median_value)
print("Standard Deviation:", sd_value)
print("Minimum:", min_value)
print("Maximum:", max_value)

       Feature  Correlation       P-value
6     totalpay     1.000000  0.000000e+00
2        cobal     0.292946  1.382315e-71
30  rollout_IN     0.072457  1.467798e-05
57  rollout_TN     0.066385  7.203619e-05
58  rollout_TX    -0.064220  1.231748e-04
..         ...          ...           ...
53  rollout_PA     0.001971  9.062963e-01
54  rollout_RI    -0.001266  9.397434e-01
35  rollout_MD     0.001014  9.517164e-01
61  rollout_VT    -0.000590  9.719055e-01
60  rollout_VA    -0.000503  9.760088e-01

[65 rows x 3 columns]
Mean: 0.015310553232362132
Mode: -0.06421980655772845
Median: -0.00838061107452592
Standard Deviation: 0.13268242133913338
Minimum: -0.06421980655772845
Maximum: 1.0


In [ ]:
df_unlabeled

,acctid,zip,cobal,bureauscr,numcalls,numrpcs,rollout_Associates,rollout_Bank_Of_Am,rollout_Chase,rollout_Chase_Bony,...,rollout_SD,rollout_TN,rollout_TX,rollout_UT,rollout_VA,rollout_VT,rollout_WA,rollout_WI,rollout_WV,rollout_WY
0,0.000000,0.913681,0.190431,0.681818,0.226368,0.000000,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000280,0.327807,0.191610,0.613636,0.130597,0.024390,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000560,0.925836,0.260134,0.681818,0.092040,0.000000,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000841,0.925836,0.085553,0.659091,0.018657,0.048780,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.001121,0.725963,0.040361,0.670455,0.263682,0.024390,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3565,0.998879,0.838080,0.178058,0.602273,0.032338,0.073171,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3566,0.999159,0.904552,0.058155,0.704545,0.113184,0.048780,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3567,0.999440,0.981742,0.160912,0.920455,0.217662,0.097561,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3568,0.999720,0.103119,0.319173,0.056818,0.016169,0.024390,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Set up features
features = df_labeled.columns.drop("totalpay")

# Set up target variable
target = ['totalpay']

# Get the data
X = df_labeled [features]
y = df_labeled [target]

# Split into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Question 2: Why did you decide to make above data preparation process regarding differnt variables

# considering the zip and state share similar meaning in terms of the account holder living location, state is used, as zip is too specific.
#from the distribution graph/scatter plot of exhitbit 2, accessor, collscr, eaglemod shows no obvious pattern, thus will be dropped from the model
# as from data dictionary, collscr cant be interpreted, thus collscr and cs would be dropped.



## Step 2: Model Building
*  Think about what is your goal for this problem. Which model you will use, classification or regression?

In [ ]:
# import your models
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

final_best_score = 0
final_model = None

for i in range(2, 8):
  param_grid = {
      'n_neighbors': list(range(1, 31)),
      'metric': ['euclidean', 'manhattan'],
  }

  # Build the KNN model
  knn = KNeighborsRegressor()

  # Perform grid search with cross-validation
  grid_search = GridSearchCV(knn, param_grid, cv=i, scoring='neg_mean_squared_error')
  grid_search.fit(X_train, y_train)

  # Get the best parameters and best score
  best_params = grid_search.best_params_
  best_score = -(grid_search.best_score_)

  if best_score > final_best_score:
    final_cv = i
    final_K = grid_search.best_params_
    final_model = grid_search

# Access the best estimator/model
best_model = final_model.best_estimator_

# Fit the model with the training set
best_model.fit(X_train, y_train)

KNeighborsRegressor(metric='manhattan', n_neighbors=30)

Question 3: How did you tune your model?

using gridsearch with different cv, metric, and n_neightbour

## Step 3: Model Evaluation
* Use testing set to evaluate your model based on your goal.

In [ ]:
# make prediction on test set
acc_cost = 800
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Make predictions on the test set
test_predictions = best_model.predict(X_test)

# Calculate mean squared error (MSE)
mse = mean_squared_error(y_test, test_predictions)

# Calculate root mean squared error (RMSE)
rmse = np.sqrt(mse)

# Calculate mean absolute error (MAE)
mae = mean_absolute_error(y_test, test_predictions)

# Calculate R-squared (coefficient of determination)
r2 = r2_score(y_test, test_predictions)

print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2):", r2)

Mean Squared Error (MSE): 1009548.9780952381
Root Mean Squared Error (RMSE): 1004.7631452711819
Mean Absolute Error (MAE): 678.2940553999377
R-squared (R2): 0.007958506525430287


#### Based on your above evaluation

In [ ]:
# How many accounts you should purchase in test set?
# How much profit you make based on those decisions?

total_revenue = 0
cost_per_account = 800  # Cost per account
revenue_per_account = test_predictions

num_accounts_to_purchase = 0
for i in range(len(revenue_per_account)):
    profit = revenue_per_account[i] - cost_per_account
    if profit > 0:
        num_accounts_to_purchase += 1
        total_revenue += revenue_per_account[i]


# Calculate the cost and total revenue
total_cost = num_accounts_to_purchase * cost_per_account

# Print the results
print("Number of accounts to purchase:", num_accounts_to_purchase)
print("Total cost:", total_cost)
print("Total revenue:", total_revenue)

Number of accounts to purchase: 474
Total cost: 379200
Total revenue: [477490.23333333]


## Step 4: Build your Final Model with all your training data
* You should use "SC Training Data.csv" to fit

In [ ]:
final_model = best_model
# Use entire data to fit the chosen model
final_model.fit(X, y)

KNeighborsRegressor(metric='manhattan', n_neighbors=30)

## Step 5: Make prediction on the test set
* Apply your model to make prediction on "SC Test Data.csv"

In [ ]:
predictions = final_model.predict(df_unlabeled)
df_unlabeled['predict_total_pay'] = predictions

In [ ]:
# Assuming 'df_unlabeled' is your unlabeled dataframe with the 'predict_total_pay' column
# Assuming 'cost_per_account' is the cost per account

# Calculate profit and determine the number of accounts to purchase
num_accounts_to_purchase = 0
total_revenue = 0
decisions = []

for i in range(len(df_unlabeled)):
    profit = df_unlabeled['predict_total_pay'][i] - cost_per_account
    if profit > 0:
        num_accounts_to_purchase += 1
        total_revenue += df_unlabeled['predict_total_pay'][i]
        decisions.append(1)
    else:
        decisions.append(0)

# Calculate the cost
total_cost = num_accounts_to_purchase * cost_per_account

# Print the updated dataframe with the 'decision' column
print(df_unlabeled)

        acctid       zip     cobal  bureauscr  numcalls   numrpcs  \
0     0.000000  0.913681  0.190431   0.681818  0.226368  0.000000   
1     0.000280  0.327807  0.191610   0.613636  0.130597  0.024390   
2     0.000560  0.925836  0.260134   0.681818  0.092040  0.000000   
3     0.000841  0.925836  0.085553   0.659091  0.018657  0.048780   
4     0.001121  0.725963  0.040361   0.670455  0.263682  0.024390   
...        ...       ...       ...        ...       ...       ...   
3565  0.998879  0.838080  0.178058   0.602273  0.032338  0.073171   
3566  0.999159  0.904552  0.058155   0.704545  0.113184  0.048780   
3567  0.999440  0.981742  0.160912   0.920455  0.217662  0.097561   
3568  0.999720  0.103119  0.319173   0.056818  0.016169  0.024390   
3569  1.000000  0.373371  0.168336   0.670455  0.110697  0.048780   

      rollout_Associates  rollout_Bank_Of_Am  rollout_Chase  \
0                    0.0                 1.0            0.0   
1                    1.0                 0.0 

Question 4: how did you make decision on whether you purchase the account or not

basically I checked if the predicted revenue - cost/ac(800) > 0



In [ ]:
# Make estimation on unlabeled data regarding totalpay, how much profit you could generate based on your prediction?
# Print the results
print("Number of accounts to purchase:", num_accounts_to_purchase)
print("Total cost:", total_cost)
print("Total revenue:", total_revenue)
print("Total profit", total_revenue - total_cost)

Number of accounts to purchase: 1719
Total cost: 1375200
Total revenue: 1744283.0000000047
Total profit 369083.00000000466


Question 5: how did you make estimate on total profit you would gain based on your decision

In [ ]:
# Export the results to a CSV file
df_decisions = pd.DataFrame({"acctid":df_unlabeled["acctid"], "decision": decisions})
df_decisions.to_csv(f"decisions_31.csv", index=False)

In [ ]:
print("DONE!")